<a href="https://colab.research.google.com/github/Fatai-soliu/Text-Classification/blob/main/Soliu_Fatai_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [52]:
url = 'https://github.com/Fatai-soliu/Text-Classification/raw/master/Scopusdataset.xlsx'

papers = pd.read_excel(url, engine='openpyxl')

In [53]:
# View first 5 rows of data

papers.head()

,Abstract,Subject
0,"In computer artificial intelligence, there is ...",Computer science and engineering
1,Artificial intelligence (AI) holds significant...,Computer science and engineering
2,The influence of Artificial Intelligence on hi...,Computer science and engineering
3,Background: Benign paroxysmal positional verti...,Computer science and engineering
4,Space-air-ground integrated networks comprise ...,Computer science and engineering


In [54]:
#Assign Label

def assign_label(subject):
    if subject == "Business":
        return 0
    elif subject == "Computer science and engineering":
        return 1
    elif subject == "Mathematics":
        return 2
    else:
        return None  # Handle other cases if needed

# Apply the function to your DataFrame
papers["label"] = papers["Subject"].apply(assign_label)


papers.tail()

,Abstract,Subject,label
14995,"In the recent past, the rise of hundreds of ‘A...",Business,0
14996,Research on AI has gained momentum in recent y...,Business,0
14997,"In contrast to point forecasting, interval for...",Business,0
14998,Worldwide investments in influencer marketing ...,Business,0
14999,The primary aim of this systematic literature ...,Business,0


In [55]:
# Drop Subject column

papers = papers.drop(columns=['Subject'], axis=1)

#print out the first five rows of papers
papers.head()

,Abstract,label
0,"In computer artificial intelligence, there is ...",1
1,Artificial intelligence (AI) holds significant...,1
2,The influence of Artificial Intelligence on hi...,1
3,Background: Benign paroxysmal positional verti...,1
4,Space-air-ground integrated networks comprise ...,1


In [56]:
# Check if there is any missing data

papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Abstract  15000 non-null  object
 1   label     15000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 234.5+ KB


# **Text preprocesssing**

In [19]:
papers1 = papers['Abstract']

papers1

0       In this paper, we propose an accountable priva...
1       It is essential to account the variability in ...
2       Unsupervised anomalous sound detection, especi...
3       Every employee in the company who deals with d...
4       The transport of a neutral solute in incompres...
                              ...                        
1137    Historically the British Trades Union Congress...
1138    ‘Emotional Weather Report’ is a song by Tom Wa...
1139    Even though the rise of the neural machine tra...
1140    This article responds to McCloud’s theoretical...
1141    Background and objectives: Visual Schema Displ...
Name: Abstract, Length: 1142, dtype: object

In [20]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
WPT = WordPunctTokenizer()

In [29]:
word_punct_token = WPT.tokenize(papers1[0])
word_tokenize = word_tokenize(papers1[0])


print(word_tokenize)

['In', 'this', 'paper', ',', 'we', 'propose', 'an', 'accountable', 'privacy', 'preserving', 'attribute-based', 'framework', ',', 'called', 'Ins-PAbAC', ',', 'that', 'combines', 'attribute', 'based', 'encryption', 'and', 'attribute', 'based', 'signature', 'techniques', 'for', 'securely', 'sharing', 'outsourced', 'data', 'contents', 'via', 'public', 'cloud', 'servers', '.', 'The', 'proposed', 'framework', 'presents', 'several', 'advantages', '.', 'First', ',', 'it', 'provides', 'an', 'encrypted', 'access', 'control', 'feature', ',', 'enforced', 'at', 'the', 'data', 'owner', "'s", 'side', ',', 'while', 'providing', 'the', 'desired', 'expressiveness', 'of', 'access', 'control', 'policies', '.', 'Second', ',', 'Ins-PAbAC', 'preserves', 'users', '’', 'privacy', ',', 'relying', 'on', 'an', 'anonymous', 'authentication', 'mechanism', ',', 'derived', 'from', 'a', 'privacy', 'preserving', 'attribute', 'based', 'signature', 'scheme', 'that', 'hides', 'the', 'users', '’', 'identifying', 'informati

In [30]:
print(word_punct_token)

['In', 'this', 'paper', ',', 'we', 'propose', 'an', 'accountable', 'privacy', 'preserving', 'attribute', '-', 'based', 'framework', ',', 'called', 'Ins', '-', 'PAbAC', ',', 'that', 'combines', 'attribute', 'based', 'encryption', 'and', 'attribute', 'based', 'signature', 'techniques', 'for', 'securely', 'sharing', 'outsourced', 'data', 'contents', 'via', 'public', 'cloud', 'servers', '.', 'The', 'proposed', 'framework', 'presents', 'several', 'advantages', '.', 'First', ',', 'it', 'provides', 'an', 'encrypted', 'access', 'control', 'feature', ',', 'enforced', 'at', 'the', 'data', 'owner', "'", 's', 'side', ',', 'while', 'providing', 'the', 'desired', 'expressiveness', 'of', 'access', 'control', 'policies', '.', 'Second', ',', 'Ins', '-', 'PAbAC', 'preserves', 'users', '’', 'privacy', ',', 'relying', 'on', 'an', 'anonymous', 'authentication', 'mechanism', ',', 'derived', 'from', 'a', 'privacy', 'preserving', 'attribute', 'based', 'signature', 'scheme', 'that', 'hides', 'the', 'users', '’

In [32]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [33]:
tokens = [x for x in word_punct_token  if x not in stop_words]
print(tokens)

['In', 'paper', ',', 'propose', 'accountable', 'privacy', 'preserving', 'attribute', '-', 'based', 'framework', ',', 'called', 'Ins', '-', 'PAbAC', ',', 'combines', 'attribute', 'based', 'encryption', 'attribute', 'based', 'signature', 'techniques', 'securely', 'sharing', 'outsourced', 'data', 'contents', 'via', 'public', 'cloud', 'servers', '.', 'The', 'proposed', 'framework', 'presents', 'several', 'advantages', '.', 'First', ',', 'provides', 'encrypted', 'access', 'control', 'feature', ',', 'enforced', 'data', 'owner', "'", 'side', ',', 'providing', 'desired', 'expressiveness', 'access', 'control', 'policies', '.', 'Second', ',', 'Ins', '-', 'PAbAC', 'preserves', 'users', '’', 'privacy', ',', 'relying', 'anonymous', 'authentication', 'mechanism', ',', 'derived', 'privacy', 'preserving', 'attribute', 'based', 'signature', 'scheme', 'hides', 'users', '’', 'identifying', 'information', '.', 'Furthermore', ',', 'proposal', 'introduces', 'accountable', 'attribute', 'based', 'signature', 

Lemmitization and stemming

In [34]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [35]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [37]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [38]:
print(lemmatizer.lemmatize(tokens[7]))

attribute


# **Train and Test Split**

In [59]:
Abstract = papers.Abstract
label = papers.label

print(Abstract)

0        In computer artificial intelligence, there is ...
1        Artificial intelligence (AI) holds significant...
2        The influence of Artificial Intelligence on hi...
3        Background: Benign paroxysmal positional verti...
4        Space-air-ground integrated networks comprise ...
                               ...                        
14995    In the recent past, the rise of hundreds of ‘A...
14996    Research on AI has gained momentum in recent y...
14997    In contrast to point forecasting, interval for...
14998    Worldwide investments in influencer marketing ...
14999    The primary aim of this systematic literature ...
Name: Abstract, Length: 15000, dtype: object


In [65]:
#Data splitting
# 30% of the data will be reserved for testing

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(papers,test_size=0.3,random_state=42)

In [66]:
train_df.shape, test_df.shape

((10500, 2), (4500, 2))

In [74]:
# Extreact the training and testing examples

train_Abstract = train_df['Abstract'].values
train_label = train_df['label'].values

test_Abstract = test_df['Abstract'].values
test_label = test_df['label'].values

In [77]:
train_Abstract.shape, test_label.shape

((10500,), (4500,))

In [80]:
#Examples of the train Abstract

train_Abstract[:5]

array(['Digital solutions are increasingly used to address “wicked problems” that are locally embedded but require global approaches. Scaling these solutions internationally is imperative for their success, but to date we know little about this process. Using a qualitative case study methodology, our paper analyzes how four digital solutions driven by the United Nations are built and how they scale internationally. These solutions address wicked problems through artificial intelligence, blockchain, and geospatial mapping, and are embedded in networks of partners which evolve during scaling to create unique ecosystem roles and configurations. We identify different ecosystem roles and find that the specific properties of digital solutions\xa0–\xa0modularity, generativity and affordances\xa0–\xa0enable either adaptation or replication during scaling. Building on these insights, we derive a typology of four different types of international scaling, which vary in their ecosystem versatility

In [81]:
train_label[:5]

array([0, 2, 2, 1, 0])

# **Biuld Model**

In [83]:
model_1 = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model_1, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_Abstract[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.30819127,  0.20944689,  0.50993955,  0.2005238 , -0.6630927 ,
        -0.24264099, -0.14509195,  0.3787548 , -0.86818445, -0.21720594,
        -0.873501  ,  0.5866086 , -0.2629904 , -0.22888619,  0.32645723,
         0.03920954,  0.23631899, -0.65621996,  0.50734794, -0.06888068,
        -0.11818752, -0.25707695,  0.9164064 , -0.47618377, -0.2929351 ,
         0.07510781, -0.231798  ,  0.31252414,  0.28091428, -0.4163451 ,
         0.20972326, -0.3079886 ,  0.1442561 , -0.6084268 , -0.58783346,
        -0.6292662 , -0.02335121, -0.36025888, -0.31005958, -0.39834014,
         0.07828069,  0.43191475, -0.30393732,  0.8066014 ,  0.2513761 ,
        -0.50885874, -0.08922997, -0.3561118 ,  0.41065055,  0.51072717],
       [ 0.72727084,  0.01603405,  0.8764496 ,  0.10964483, -0.9042398 ,
        -0.6236467 , -0.274023  ,  0.14207155, -0.6579888 , -0.11956891,
        -0.27730098,  0.438937  , -0.19402303, -0.52480245,  0.3995676 ,
 

In [85]:
model_2 = "https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2"
hub_layer = hub.KerasLayer(model_2, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_Abstract[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[-0.03246222,  0.23952438,  0.6666062 ,  0.0866725 , -0.9752053 ,
        -0.3768316 , -0.08369859,  0.351678  , -1.0651877 , -0.157682  ,
        -0.80772424,  0.6041593 , -0.29249018, -0.23391913,  0.27230182,
         0.08928214,  0.24121727, -0.8200853 ,  0.6312705 ,  0.07431545,
        -0.03016189, -0.32984394,  1.1177435 , -0.60634196, -0.43697554,
         0.37006617, -0.26711345,  0.3651609 ,  0.33484185, -0.48895782,
         0.11570848, -0.39807752,  0.15745302, -0.6970522 , -0.76136774,
        -0.67240137,  0.03430431, -0.45808986, -0.22112237, -0.46324712,
        -0.07216966,  0.43243963, -0.06433047,  0.9216442 ,  0.22217512,
        -0.716571  , -0.14210647, -0.49654132,  0.5683729 ,  0.57151777],
       [ 0.5108946 ,  0.06035847,  0.90636826,  0.11856593, -1.1565678 ,
        -0.6671171 , -0.17987193,  0.23133308, -0.79685366, -0.09055639,
        -0.31400445,  0.49605408, -0.23103005, -0.6169743 ,  0.39336246,
 

In [86]:
model_3 = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model_3, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_Abstract[:3])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[ 1.54860580e+00, -1.02506056e-01,  3.40369314e-01,
        -1.91790685e-01, -3.56966674e-01,  1.45241497e-02,
         2.27144435e-01, -5.71509339e-02, -3.12246174e-01,
         5.12111068e-01,  1.97600961e-01, -1.51935637e-01,
        -2.04322472e-01,  1.90892816e-03, -5.25328934e-01,
        -2.18335807e-01, -2.69521534e-01, -4.70605195e-02,
        -4.43803161e-01,  1.11671865e+00,  2.19564855e-01,
        -1.30544692e-01, -3.85573544e-02,  1.44983634e-01,
         2.26829574e-01,  5.49685471e-02,  7.40156174e-02,
         4.39704776e-01,  5.13541251e-02,  1.63469344e-01,
        -8.27084035e-02, -1.06585838e-01,  1.33365080e-01,
        -2.12458223e-02, -2.55759656e-02, -1.37422264e-01,
         6.21008314e-02, -2.16192856e-01, -1.74775105e-02,
         1.35856241e-01,  5.49127050e-02, -2.56466061e-01,
        -2.10632049e-02,  1.49504766e-01, -9.98005550e-03,
        -7.98494816e-02,  2.60071568e-02, -5.01104534e-01,
      

In [88]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124644769 (475.48 MB)
Trainable params: 124644769 (475.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
